# CUDA tensor network contraction demo

## Requirements
- The system must have a CUDA GPU available.

In [1]:
using Tenet
using EinExprs
using Adapt
using CUDA
using BenchmarkTools

┌ Warning: You are using a non-official build of Julia. This may cause issues with CUDA.jl.
│ Please consider using an official build from https://julialang.org/downloads/.
└ @ CUDA /home/bsc/bsc021386/.julia/packages/CUDA/75aiI/src/initialization.jl:180


Create a random tensor network and find its contraction path:

In [2]:
# Initialize random tensor network
regularity = 6
ntensors = 10
tn = rand(TensorNetwork, ntensors, regularity)
path = einexpr(tn; optimizer=Exhaustive())

SizedEinExpr{Symbol}(EinExpr{Symbol}(Symbol[], EinExpr{Symbol}[EinExpr{Symbol}([:C, :a, :N, :I], EinExpr{Symbol}[]), EinExpr{Symbol}([:C, :a, :N, :I], EinExpr{Symbol}[EinExpr{Symbol}([:C, :N, :G, :Z, :U, :R, :E, :X, :S], EinExpr{Symbol}[]), EinExpr{Symbol}([:a, :I, :G, :Z, :U, :R, :E, :X, :S], EinExpr{Symbol}[EinExpr{Symbol}([:B, :D, :G, :Z], EinExpr{Symbol}[]), EinExpr{Symbol}([:B, :a, :I, :D, :U, :R, :E, :X, :S], EinExpr{Symbol}[EinExpr{Symbol}([:I, :D, :E, :O, :M], EinExpr{Symbol}[]), EinExpr{Symbol}([:B, :a, :U, :R, :X, :S, :O, :M], EinExpr{Symbol}[EinExpr{Symbol}([:B, :V, :P, :a, :W, :A], EinExpr{Symbol}[EinExpr{Symbol}([:B, :V, :J, :b, :c, :P], EinExpr{Symbol}[]), EinExpr{Symbol}([:J, :b, :c, :a, :W, :A], EinExpr{Symbol}[])]), EinExpr{Symbol}([:V, :P, :U, :R, :X, :S, :W, :A, :O, :M], EinExpr{Symbol}[EinExpr{Symbol}([:P, :X, :T, :L, :H], EinExpr{Symbol}[]), EinExpr{Symbol}([:V, :U, :R, :S, :W, :A, :O, :M, :T, :L, :H], EinExpr{Symbol}[EinExpr{Symbol}([:R, :S, :A, :K, :M, :d, :Y, :T

Transform the tensors' data types to `CuArray`s:

In [3]:
cudatn = adapt(CuArray, tn)

TensorNetwork (#tensors=10, #inds=30)

Benchmark CUDA tensor network contraction:

In [4]:
@benchmark CUDA.@sync contract(cudatn; path)

BenchmarkTools.Trial: 3371 samples with 1 evaluation.
 Range (min … max):  1.324 ms …  16.946 ms  ┊ GC (min … max): 0.00% … 55.47%
 Time  (median):     1.389 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.469 ms ± 940.462 μs  ┊ GC (mean ± σ):  3.02% ±  4.18%

         ▁▅█▇▅▃▁                                               
  ▂▃▆▇█▇████████▇▅▅▄▃▃▂▂▂▂▂▂▁▁▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▂▂▂▂ ▃
  1.32 ms         Histogram: frequency by time        1.71 ms <

 Memory estimate: 258.63 KiB, allocs estimate: 4418.

Benchmark regular tensor network contraction:

In [5]:
@benchmark CUDA.@sync contract(tn; path)

BenchmarkTools.Trial: 40 samples with 1 evaluation.
 Range (min … max):   65.622 ms … 261.489 ms  ┊ GC (min … max):  3.43% … 74.33%
 Time  (median):      97.514 ms               ┊ GC (median):     2.04%
 Time  (mean ± σ):   130.156 ms ±  69.544 ms  ┊ GC (mean ± σ):  39.14% ± 29.56%

  ▄▄▁   █▄ █ █ ▁    ▁                                    ▁▄ ▁    
  ███▁▁▆██▁█▆█▁█▁▁▁▁█▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆▁▁▁▆██▁█▁▆ ▁
  65.6 ms          Histogram: frequency by time          261 ms <

 Memory estimate: 222.25 MiB, allocs estimate: 2835.